In [1]:
"""
NMT Abnormal Dataset
"""

import glob, os
import numpy as np
from tqdm import tqdm
import torch

from IPython.display import clear_output

#from src.model import EEGModel
from src.read_data import build_data
from src.make_features import train_test
#from src.train import train_model, print_acc

def norm_adj(train_graphs, test_graphs):
    for i in range(train_graphs.shape[0]):
        for j in range(train_graphs.shape[1]):
                min_ = (train_graphs[i, j, :, :]).min()
                max_ = (train_graphs[i, j, :, :]).max()
                train_graphs[i, j, :,  :] = (train_graphs[i, j, :,  :] - min_)/(max_ - min_)
                
    for i in range(test_graphs.shape[0]):
        for j in range(test_graphs.shape[1]):
            min_ = (test_graphs[i, j, :, :]).min()
            max_ = (test_graphs[i, j, :, :]).max()
            test_graphs[i, j, :, :] = (test_graphs[i, j, :,  :] - min_)/(max_ - min_)
            
    return train_graphs, test_graphs


def norm_adj(train_graphs, test_graphs):
    for i in range(train_graphs.shape[2]):
        for j in range(train_graphs.shape[3]):
            if train_graphs[:, :, i, j].sum != 0:
                min_ = (train_graphs[:, :, i, j]).min()
                max_ = (train_graphs[:, :, i, j]).max()
                train_graphs[:, :, i,  j] = (train_graphs[:, :, i,  j] - min_)/(max_ - min_)
                test_graphs[:, :, i, j] = (test_graphs[:, :, i,  j] - min_)/(max_ - min_)
            if i == j:
                train_graphs[:, :, i, j] = 1
            
    return train_graphs, test_graphs

def norm_adj(train_graphs, test_graphs):
    for i in range(train_graphs.shape[0]):
        for j in range(train_graphs.shape[1]):
                min_ = (train_graphs[i, j, :, :]).min()
                max_ = (train_graphs[i, j, :, :]).max()
                train_graphs[i, j, :,  :] = (train_graphs[i, j, :,  :] - min_)/(max_ - min_)
                
    for i in range(test_graphs.shape[0]):
        for j in range(test_graphs.shape[1]):
            min_ = (test_graphs[i, j, :, :]).min()
            max_ = (test_graphs[i, j, :, :]).max()
            test_graphs[i, j, :, :] = (test_graphs[i, j, :,  :] - min_)/(max_ - min_)
            
    return train_graphs, test_graphs
    
SAVED_DATA = True
SAVED_FEATURES = True
CONN_TYPES = ["plv"]#, "ciplv", "coh", "pc"]

# Files names on disk
files, train_files, test_files = [], [], []
for file in glob.glob("/Users/mohamedr/projects/limited_data/graphs/bands/dynamic_graphs/nmt/*/*/*/*.edf"):
    files.append(file)
    if "train" in file:
        train_files.append(file)
    else:
        test_files.append(file)
print(len(files))
np.random.shuffle(train_files)
print(len(train_files), len(test_files))

for conn in CONN_TYPES:    
    print(conn)
    if SAVED_FEATURES:
        train_X = np.load("../saved_npy_nmt/features/train_X.npy") #samples, #epochs, #channels, #time points)
        train_y = np.load("../saved_npy_nmt/features/train_y.npy")
        train_graphs = np.load("../saved_npy_nmt/features/train_graphs_"+conn+".npy")
        test_y = np.load("../saved_npy_nmt/features/test_y.npy")
        test_X = np.load("../saved_npy_nmt/features/test_X.npy")
        test_graphs = np.load("../saved_npy_nmt/features/test_graphs_"+conn+".npy")
        train_X = np.moveaxis(train_X, 1, -1)
        test_X = np.moveaxis(test_X, 1, -1)
        train_graphs, test_graphs = norm_adj(train_graphs, test_graphs)
        #train_X, test_X = standardize_data(train_X, test_X)
        
    else:
        if SAVED_DATA:
            # Saved arrays on disk
            train_X_files = ["../saved_npy_nmt/features/train_X"+str(i)+".npy" for i in range(0, len(train_files), 100)]
            train_y_files = ["../saved_npy_nmt/features/train_y"+str(i)+".npy" for i in range(0, len(train_files), 100)]
            train_X = np.vstack([np.load(file).astype(np.float16) for i, file in enumerate(train_X_files)])
            train_y = np.vstack([np.load(file).astype(np.float16) for i, file in enumerate(train_y_files)])
            test_X = np.load("../saved_npy_nmt/features/test_X.npy").astype(np.float16)
            test_y = np.load("../saved_npy_nmt/features/test_y.npy").astype(np.float16)
            
        else:
            num_windows = 100
            window_size = 200
            for i in range(0, len(train_files), 100):
                train_X, train_y = build_data(train_files[i:i+100], use_windows=False, window_size=window_size, 
                                              num_windows=num_windows, dataset = "nmt")
                np.save("../saved_npy_nmt/features/train_X"+str(i)+".npy", train_X.astype(np.float16))
                np.save("../saved_npy_nmt/features/train_y"+str(i)+".npy", train_y.astype(np.float16))
            test_X, test_y = build_data(test_files, use_windows=False, window_size=window_size, 
                                        num_windows=num_windows, dataset = "nmt")
            np.save("../saved_npy_nmt/features/test_X.npy", test_X.astype(np.float16))
            np.save("../saved_npy_nmt/features/test_y.npy", test_y.astype(np.float16))
            clear_output()
    
            train_X_files = ["../saved_npy_nmt/features/train_X"+str(i)+".npy" for i in range(0, len(train_files), 100)]
            train_y_files = ["../saved_npy_nmt/features/train_y"+str(i)+".npy" for i in range(0, len(train_files), 100)]
            train_X = np.vstack([np.load(file) for file in train_X_files])
            train_y = np.vstack([np.load(file) for file in train_y_files])
            test_X = np.load("../saved_npy_nmt/features/test_X.npy")
            test_y = np.load("../saved_npy_nmt/features/test_y.npy")

        np.save("../saved_npy_nmt/features/train_X.npy", train_X)
        np.save("../saved_npy_nmt/features/train_y.npy", train_y)
        np.save("../saved_npy_nmt/features/test_y.npy", test_y)
        np.save("../saved_npy_nmt/features/test_X.npy", test_X)
        print("make features")
        _ , train_graphs, _, _ , test_graphs, _ = train_test(train_X=train_X, 
                                                             test_X=test_X, 
                                                             train_y=train_y, 
                                                             test_y=test_y, 
                                                             method=conn)
        
        np.save("../saved_npy_nmt/features/train_X.npy", train_X)
        np.save("../saved_npy_nmt/features/train_y.npy", train_y)
        np.save("../saved_npy_nmt/features/train_graphs_"+conn+".npy", train_graphs)
        np.save("../saved_npy_nmt/features/test_y.npy", test_y)
        np.save("../saved_npy_nmt/features/test_X.npy", test_X)
        np.save("../saved_npy_nmt/features/test_graphs_"+conn+".npy", test_graphs)

2417
2232 185
plv


In [4]:
abnormals = 0
normals = 0
for file in test_files:
    if "abnormal" in file:
        abnormals+=1
    else:
        normals+=1

abnormals, normals

(90, 95)